In [1]:
import pathlib
import json
import pandas as pd
from yaml import load, dump, SafeLoader
import seaborn as sns
import matplotlib.pyplot as plt

# set default style and size of plots
sns.set_theme(style="whitegrid")
sns.set(rc={'figure.figsize':(11.7,8.27)})


In [2]:
folder = pathlib.Path().parent.resolve() / "psql-copy"

In [3]:
results = []
for path in folder.glob("*/**/results.json"):
    path_folder = path.parent
    config_path = path_folder / ".hydra" / "config.yaml"
    with open(config_path, "r") as f:
        config = load(f, Loader=SafeLoader)
    config = {k: v for d in config.values() for k, v in d.items()}
    with open(path, "r") as f:
        result = json.load(f)
    
    results.append({**config, **result, "path": path_folder.resolve()})

In [4]:
df = pd.DataFrame(results)

df.loc[lambda x: x.host == "rc1a-t7a7rebv6fa9ramf.mdb.yandexcloud.net", "db"] = "clickhouse"
df.loc[lambda x: x.host == "rc1a-c815b7chps4w9cod.mdb.yandexcloud.net", "db"] = "postgres"

In [5]:
df.groupby(["host", "db", "segments"])[["time", "time_per_segment"]].agg(["mean", "std", "min"])

time  \
                                                                     mean   
host                                      db         segments               
localhost                                 postgres   100         0.100668   
                                                     10000       7.963891   
                                                     100000     84.121420   
rc1a-c815b7chps4w9cod.mdb.yandexcloud.net postgres   100         2.841141   
                                                     10000     249.867179   
rc1a-t7a7rebv6fa9ramf.mdb.yandexcloud.net clickhouse 100         2.710285   
                                                     10000      37.827020   
                                                     100000    158.072280   

                                                                          \
                                                                     std   
host                                      db         segments              
localhost                                 postgres   100        0.005211   
                                                     10000      0.588200   
                                                     100000     2.809741   
rc1a-c815b7chps4w9cod.mdb.yandexcloud.net postgres   100        1.493975   
                                                     10000     33.804937   
rc1a-t7a7rebv6fa9ramf.mdb.yandexcloud.net clickhouse 100        1.327665   
                                                     10000     23.866672   
                                                     100000    21.938008   

                                                                           \
                                                                      min   
host                                      db         segments               
localhost                                 postgres   100         0.096983   
                                                     10000       7.387329   
                                                     100000     80.878938   
rc1a-c815b7chps4w9cod.mdb.yandexcloud.net postgres   100         1.586748   
                                                     10000     219.406552   
rc1a-t7a7rebv6fa9ramf.mdb.yandexcloud.net clickhouse 100         1.617250   
                                                     10000      18.526035   
                                                     100000    132.745965   

                                                              time_per_segment  \
                                                                          mean   
host                                      db         segments                    
localhost                                 postgres   100              0.001007   
                                                     10000            0.000796   
                                                     100000           0.000841   
rc1a-c815b7chps4w9cod.mdb.yandexcloud.net postgres   100              0.028411   
                                                     10000            0.024987   
rc1a-t7a7rebv6fa9ramf.mdb.yandexcloud.net clickhouse 100              0.027103   
                                                     10000            0.003783   
                                                     100000           0.001581   

                                                                         \
                                                                    std   
host                                      db         segments             
localhost                                 postgres   100       0.000052   
                                                     10000     0.000059   
                                                     100000    0.000028   
rc1a-c815b7chps4w9cod.mdb.yandexcloud.net postgres   100       0.014940   
                                                     10000     0.003380   
rc1a-t7a7rebv6fa9ramf.mdb.yandexc